# **Arxiv metadata Analytics with PySpark RDD: JSON case study**

In [1]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=feb7045da699a06536d4139da829acce9b50f16b2eed258c2c03de2e1ceb81f2
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [3]:
# Initializing Spark
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Arxiv").setMaster("local[*]")
sc = SparkContext(conf = conf)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Read and Load Data to Spark
# Data source: https://www.kaggle.com/Cornell-University/arxiv/version/62

import json

# making 100 partitions of the data

data = sc.textFile("/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json", 100)
rdd = data.map(lambda x: json.loads(x))

rdd.persist()

PythonRDD[4] at RDD at PythonRDD.scala:53

In [7]:
# Check the number of parallelism and partitions:
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

2
100


## Question 1: Count elements

In [8]:
rdd.count()

2011231

## Question 2: Get the first two records


In [9]:
rdd.take(2)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

## Question 3: Get all attributes


In [10]:
rdd.flatMap(lambda x: x.keys()).distinct().collect()

['authors',
 'comments',
 'title',
 'id',
 'journal-ref',
 'versions',
 'submitter',
 'categories',
 'update_date',
 'authors_parsed',
 'report-no',
 'license',
 'abstract',
 'doi']

## Question 4: Get the name of the licenses

In [12]:
rdd.map(lambda x: x['license']).distinct().collect()

[None,
 'http://creativecommons.org/licenses/publicdomain/',
 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/3.0/',
 'http://creativecommons.org/licenses/by/3.0/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/publicdomain/zero/1.0/',
 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'http://creativecommons.org/licenses/by-sa/4.0/']

## Question 5: Get the shortest and the longest titles

In [13]:
short_title = rdd.map(lambda x: x['title']).min()
long_title = rdd.map(lambda x: x['title']).max()

print(short_title)
print(long_title)

!-Graphs with Trivial Overlap are Context-Free
Weyl formula for the negative dissipative eigenvalues of Maxwell's
  equations


In [14]:
short_title = rdd.map(lambda x: x['title']).reduce(lambda x,y: x if x < y else y)
long_title = rdd.map(lambda x: x['title']).reduce(lambda x,y: x if x > y else y)

print(short_title)
print(long_title)

!-Graphs with Trivial Overlap are Context-Free
Weyl formula for the negative dissipative eigenvalues of Maxwell's
  equations


## Question 6: Find abbreviations with 5 or more letters in the abstract

In [15]:
import re

def get_abbrivations(line):
  result = re.search(r"\(([A-Za-z][^_/\\<>]{5,})\)", line)
  if result:
    return result.group(1)

In [17]:
rdd.filter(lambda x:get_abbrivations(x['abstract'])).count()

702869

## Question 7: Get the number of archive records per month ('update_date' attribute)

In [18]:
import datetime

def extract_month(datein):
  d = datetime.datetime.strptime(datein, '%Y-%m-%d')
  return d.month

extract_month('2024-10-17')

10

In [19]:
rdd.map(lambda x: (extract_month(x['update_date']),1)).reduceByKey(lambda x,y: x+y).collect()

[(1, 134247),
 (2, 116948),
 (3, 126458),
 (4, 117126),
 (5, 296587),
 (6, 191746),
 (7, 122649),
 (8, 138469),
 (9, 138978),
 (10, 197755),
 (11, 297963),
 (12, 132305)]

## Question 8: Get the average number of pages

In [21]:
def get_pages(line):
  search = re.findall(r"\d+ pages", line)
  if search:
    return int(search[0].split(" ")[0])
  else:
    return 0


get_pages('there are 100 pages')

100

In [25]:
rdd_average = rdd.map(lambda x: get_pages(x['comments'] if x['comments'] != None else "None"))
rdd_average = rdd_average.filter(lambda x: x != 0)

average_counter = rdd_average.count()
average_summation = rdd_average.reduce(lambda x,y: x+y)
print(average_counter)
print(average_summation)
print("Average of pages", (average_summation/average_counter))

1184075
21139516
Average of pages 17.85319004286046
